In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#PREPARE TRAINING SENTENCES
! pip install ekphrasis
! pip install emoji

In [ ]:
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
import pandas as pd
import numpy as np

### Read Datasets

In [ ]:
# Specify the path to your CSV file
csv_file = '/content/drive/Shareddrives/HLT/dataset/development/haspeede2-task-AB/merged_training_textual-haspeede2.csv'
dev_data = pd.read_csv(csv_file)
dev_data.head()

,anonymized_text,label
0,@unknown Con tutte le denunce che si sta becca...,1
1,#PRESCRIZIONE : I (t)re magi hanno trovato l’a...,0
2,Il #m5s ha votato contro le #unionicivili addu...,1
3,La #Lega e il #m5s stanno dando a bere all'opi...,0
4,Che cosa cambia questa #legge? Caro @667517147...,0


In [ ]:
dev_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12437 entries, 0 to 12436
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   anonymized_text  12437 non-null  object
 1   label            12437 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 194.5+ KB


### Data prerpocessing pipeline

In [ ]:
! python /usr/local/lib/python3.10/dist-packages/ekphrasis/tools/generate_stats.py --input /content/drive/Shareddrives/HLT/dataset/italian_corpus.txt --name tweet_ita --ngrams 2 --mincount 70 30


computing statistics for file:  /content/drive/Shareddrives/HLT/dataset/italian_corpus.txt
100% 10013615/10013615 [06:59<00:00, 23855.08it/s]

Writing 1-grams...
entries:326,541	-	tokens:174,118,463
writing stats to file /usr/local/lib/python3.10/dist-packages/ekphrasis/tools/../stats/tweet_ita/counts_1grams.txt
Writing 2-grams...
entries:5,653,502	-	tokens:174,118,463
writing stats to file /usr/local/lib/python3.10/dist-packages/ekphrasis/tools/../stats/tweet_ita/counts_2grams.txt


In [ ]:
text_processor = TextPreProcessor (
    # terms that will be normalized
    normalize=['url', 'email', 'user', 'percent', 'money', 'phone', 'time', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag"} ,
    fix_html=True ,  # fix HTML tokens
    #all_caps_tag="wrap", # how to wrap the capitalized words
    unpack_hashtags=True ,  # perform word segmentation on hashtags
    fix_text=True, #choose if you want to fix bad unicode terms and html entities
    segmenter="/usr/local/lib/python3.10/dist-packages/ekphrasis/stats/tweet_ita", # define the statistics of what corpus you would like to use
    corrector="/usr/local/lib/python3.10/dist-packages/ekphrasis/stats/tweet_ita", # same as before
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize
)

/usr/local/lib/python3.10/dist-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading /usr/local/lib/python3.10/dist-packages/ekphrasis/stats/tweet_ita - 1grams ...
generating cache file for faster loading...
reading ngrams /usr/local/lib/python3.10/dist-packages/ekphrasis/stats/tweet_ita/counts_1grams.txt
Reading /usr/local/lib/python3.10/dist-packages/ekphrasis/stats/tweet_ita - 2grams ...
generating cache file for faster loading...
reading ngrams /usr/local/lib/python3.10/dist-packages/ekphrasis/stats/tweet_ita/counts_2grams.txt
Reading /usr/local/lib/python3.10/dist-packages/ekphrasis/stats/tweet_ita - 1grams ...


/usr/local/lib/python3.10/dist-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [ ]:
def clean_disguised_bad_words(text):
    text = " " + text + " "
    #print(text)
    text = re.sub(r' vaff[xx]+lo ', ' vaffanculo ', text) # no need?
    text = re.sub(r' vaffan[xx**]+lo ', ' vaffanculo ', text) # no need?
    text = re.sub(r' aff[xx]+lo ', ' affanculo ', text)
    text = re.sub(r' affan[xx**]+lo ', ' affanculo ', text)
    text = re.sub(r' c[glo.x*@%#$^]+i ', ' coglioni ', text)
    text = re.sub(r' c[glo.x*@%#$^]+e ', ' coglione ', text)
    text = re.sub(r' ca\*\*o ', ' cazzo ', text, flags=re.IGNORECASE)
    text = re.sub(r' c[az.x*@%#$^]+ro ', ' cazzaro ', text)
    text = re.sub(r' inc[az.x*@%#$^]+to ', ' incazzato ', text)
    text = re.sub(r' c[az.x*@%#$^]+te ', ' cazzate ', text)
    text = re.sub(r' c[az.x*@%#$^]+i ', ' cazzi ', text)
    text = re.sub(r' c[az.x*@%#$^]+o ', ' cazzo ', text)
    text = re.sub(r' cazxxx ', ' cazzate ', text)
    text = re.sub(r' CAZO ', ' CAZZO ', text)
    text = re.sub(r' c[az.x*@%#$^]+one ', ' cazzone ', text)
    text = re.sub(r' m\*\*\*a ', ' merda ', text, flags=re.IGNORECASE)
    text = re.sub(r' m[erd.x*@%#$^]+@ ', ' merda ', text)
    text = re.sub(r' m[erd.x*@%#$^]+e ', ' merde ', text)
    text = re.sub(r' c[.x*@%#$^]+lo ', ' culo ', text)
    text = re.sub(r' culx ', ' culo ', text)
    text = re.sub(r' p[u.x*@%#$^]+ana ', ' puttana ', text)
    text = re.sub(r' p[tu.x*@%#$^]+e ', ' puttane ', text)
    text = re.sub(r' t[.x*@%#$^]+a ', ' troia ', text)
    text = re.sub(r' t[.x*@%#$^]+e ', ' troie ', text)
    text = re.sub(r' s[.x*@%#$^]+o ', ' stronzo ', text)
    text = re.sub(r' stronz[.x*@%#$^]+a ', ' stronzata ', text)
    text = re.sub(r' s[.x*@%#$^]+i ', ' stronzi ', text)
    text = re.sub(r' min[*t]+a ', ' minchiata ', text)
    return text

In [ ]:
from itertools import groupby

def deleteDuplicate (riga):
    parole = riga.split()
    stringa = ""
    for a in parole:
        parola = a
        a = [list(g) for k, g in groupby(a)]
        vocali = ['a','e','i','o','u','y']

        for idx,_ in enumerate(a):
            if idx == 0:
                stringa += a[idx][0]
            elif idx == len(a)-1:
                stringa += a[idx][0]
            elif a[idx][0] in vocali and (parola not in ['alcool', 'preesame']):
                stringa += a[idx][0]
            elif len(a[idx]) == 1:
                stringa += a[idx][0]
            elif len (a[idx]) >= 2:
                stringa += a[idx][0]
                stringa += a[idx][1]
        stringa =  stringa + " "
    return(stringa)

In [ ]:
import re
import emoji

def alberto_preprocessing(sentences):

  examples = []

  for s in sentences:
      #s = s.lower() #lowercasing
      s = re.sub(r'\[URL\]', '<url>', s) # URL Replacement:
      s = re.sub(r'#[Mm]5[Ss]', '#movimentocinquestelle', s)
      s = re.sub(r'[Mm]5[Ss]', 'movimentocinquestelle', s)
      #s = emoji.demojize(s, language='it') # Converts emojis into a textual representation
      s = clean_disguised_bad_words(s) # clean up disguised bad words
      s = deleteDuplicate(s)
      s = str(" ".join(text_processor.pre_process_doc(s)))
      s = re.sub(r"[^a-zA-ZÀ-ú</>!?\d\s]", ' ', s) # Removes characters that are not letters,digits, HTML-like tags,punctuation marks (! and ?), spaces, or Unicode characters in the range \U00010000-\U0010ffff.
      s = re.sub(r'\d+', '<number>', s)
      s = re.sub(r"\s+", ' ', s) #  Replaces multiple consecutive whitespace characters with a single space
      # s = re.sub(r'(\w)\1{2,}',r'\1\1', s) Reduce sequences where the same character is repeated three or more times to just two repetitions (e.g., "hellooooo" to "helloo").
      s = re.sub ( r'^\s' , '' , s ) # removes leading whitespace characters
      s = re.sub ( r'\s$' , '' , s ) # removes trailing whitespace characters
      #print(s)
      examples.append(s)
  return examples

### DEVELOPMENT

In [ ]:
sentences = dev_data.iloc[:,0]

In [ ]:
dev_data['anonymized_text'] = alberto_preprocessing(sentences)
dev_data

,anonymized_text,label
0,<user> con tutte le denunce che si sta beccand...,1
1,<hashtag> prescrizione </hashtag> i t re magi ...,0
2,il <hashtag> movimento cinque stelle </hashtag...,1
3,la <hashtag> lega </hashtag> e il <hashtag> mo...,0
4,che cosa cambia questa <hashtag> legge </hasht...,0
...,...,...
12432,gli stati nazionali devono essere pronti a rin...,0
12433,il ministro dell interno della germania <hasht...,0
12434,<hashtag> salvini </hashtag> in italia troppi ...,0
12435,<user> <user> chi giubila in buona fede non ha...,0


In [ ]:
from pathlib import Path
filepath = Path('/content/drive/Shareddrives/HLT/dataset/development/alberto_preprocessig/alberto_preprocessing_training_textual_haspeede2.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
dev_data.to_csv(filepath,index=False)